# H Corona sim test

Import the relevant packages:

In [ ]:
import numpy as np

In [ ]:
%cd /home/mike/Documents/Mars/3D_planetary_RT_model/python
%bookmark corona_model
from py_corona_sim_gpu import Pyobservation_fit
#from py_corona_sim_cpu import Pyobservation_fit
%cd /home/mike

In [ ]:
import idl_colorbars

In [ ]:
from scipy.spatial.transform import Rotation as R

In [ ]:
posdir=np.array([0,-1,0])
posdir=posdir/np.linalg.norm(posdir)
pos=10*3395e5*posdir

right_dir=np.array([1,0,0])
up_dir=np.cross(posdir,right_dir)
up_dir=up_dir/np.linalg.norm(up_dir)
right_dir=-np.cross(posdir,up_dir)
right_dir=right_dir/np.linalg.norm(right_dir)

center_dir=-posdir
angle=60
npix=600
lookangles=np.linspace(-angle*np.pi/180.,angle*np.pi/180.,npix)

In [ ]:
def lookdir(az,ax):
    angle1=R.from_rotvec(-az*up_dir).as_matrix()
    angle2=R.from_rotvec(ax*right_dir).as_matrix()
    return np.array([*map(lambda x:np.matmul(x,center_dir),
                          [*map(np.matmul,angle1,angle2)])])


In [ ]:
anglepairs=np.array([([ax],[az]) for az in lookangles for ax in lookangles])
all_lookangles=lookdir(anglepairs[:,0],anglepairs[:,1])
all_pos = np.array([pos for i in all_lookangles])

In [ ]:
pycorona = Pyobservation_fit()
pycorona.add_observation(all_pos,all_lookangles)
pycorona.set_g_factor([1.22848e-3,2.52e-6])
import time
time.sleep(1)

In [ ]:
pycorona.corona_model_git_hash()

In [ ]:
test_nH = 5e5
test_Texo = 200
%time pycorona.generate_source_function(test_nH,test_Texo)
%time brightness = pycorona.brightness()
%time species_col_dens = pycorona.species_col_dens()

In [ ]:
absorption = np.exp(-np.where(pycorona.tau_absorber_final() == -1, 1e5, pycorona.tau_absorber_final()))

In [ ]:
np.max(absorption*pycorona.tau_species_final(),axis=1)

In [ ]:
brightness.max()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
from matplotlib.colors import LogNorm,PowerNorm
# C = some matrix
f, ax = plt.subplots(1,dpi=250)

import copy
my_cmap = copy.copy(idl_colorbars.getcmap(109)) # copy the default cmap
my_cmap.set_bad((0,0,0))

im = ax.imshow(brightness[0].reshape((npix,npix)), cmap=my_cmap, norm=PowerNorm(0.5,vmin=0.0, vmax=12, clip=True))
ax.set_title('Singlet CFR Treatment')
t = [0,5,10,15,20]
#im = ax.imshow(brightness.reshape((npix,npix)), cmap=my_cmap, norm=LogNorm(vmin=0.1, vmax=20, clip=True))
#t = [0.1,0.5,1,5,10,20]

ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel(r'H Ly $\alpha$'+'\n[kR]', rotation=0, labelpad=25);

This will plot Lyman beta if observation_fit computes it:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
from matplotlib.colors import LogNorm,PowerNorm
# C = some matrix
f, ax = plt.subplots(dpi=250)

import copy
my_cmap = copy.copy(idl_colorbars.getcmap(114)) # copy the default cmap
my_cmap.set_bad((0,0,0))

im = ax.imshow(brightness[1].reshape((npix,npix))*1000, cmap=my_cmap, norm=PowerNorm(0.5,vmin=0.0, vmax=80, clip=True))
#t = [0,50,100]
#im = ax.imshow(brightness.reshape((npix,npix)), cmap=my_cmap, norm=LogNorm(vmin=0.1, vmax=20, clip=True))
#t = [0.1,0.5,1,5,10,20]

ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel(r'H Ly $\beta$'+'\n[R]', rotation=0, labelpad=25);

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
from matplotlib.colors import LogNorm,PowerNorm
# C = some matrix
f, ax = plt.subplots(1,dpi=250)

import copy
my_cmap = copy.copy(idl_colorbars.getcmap(49)) # copy the default cmap
my_cmap.set_bad((0,0,0))

im = ax.imshow(species_col_dens[0].reshape((npix,npix)), cmap=my_cmap, norm=LogNorm())#PowerNorm(0.5, vmin=0.0, vmax=species_col_dens.max(), clip=True))
#t = [0,5,10,15,20]
#im = ax.imshow(brightness.reshape((npix,npix)), cmap=my_cmap, norm=LogNorm(vmin=0.1, vmax=20, clip=True))
#t = [0.1,0.5,1,5,10,20]

ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel(r'H col. dens.'+'\n'+r'[cm$^{-2}$]', rotation=0, labelpad=35);

In [ ]:
np.log10(2*test_nH*3595e5)

In [ ]:
np.max(species_col_dens)

# Deuterium

In [ ]:
test_nD = test_nH /100. #5.5e3
%time pycorona.generate_source_function(test_nD,test_Texo, deuterium=True)
%time D_brightness = pycorona.D_brightness()
%time D_col_dens = pycorona.D_col_dens()

In [ ]:
absorption = np.exp(-np.where(pycorona.tau_absorber_final() == -1, 1e5, pycorona.tau_absorber_final()))

In [ ]:
np.max(absorption*pycorona.tau_D_final(),axis=1)

In [ ]:
brightness.max()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
from matplotlib.colors import LogNorm,PowerNorm
# C = some matrix
f, ax = plt.subplots(1,dpi=250)

import copy
my_cmap = copy.copy(idl_colorbars.getcmap(109)) # copy the default cmap
my_cmap.set_bad((0,0,0))

im = ax.imshow(D_brightness[0].reshape((npix,npix)), cmap=my_cmap, norm=PowerNorm(0.5,
                                                                                  vmin=0.0, 
                                                                                  vmax=D_brightness.max(), # 1.6, #
                                                                                  clip=True))
t = [0,5,10,15,20]
#im = ax.imshow(brightness.reshape((npix,npix)), cmap=my_cmap, norm=LogNorm(vmin=0.1, vmax=20, clip=True))
#t = [0.1,0.5,1,5,10,20]

ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel(r'D Ly $\alpha$'+'\n[kR]', rotation=0, labelpad=25);

This will plot Lyman beta if observation_fit computes it:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
from matplotlib.colors import LogNorm,PowerNorm
# C = some matrix
f, ax = plt.subplots(dpi=250)

import copy
my_cmap = copy.copy(idl_colorbars.getcmap(114)) # copy the default cmap
my_cmap.set_bad((0,0,0))

im = ax.imshow(D_brightness[1].reshape((npix,npix))*1000, cmap=my_cmap, norm=PowerNorm(0.5,vmin=0.0, vmax=1000*D_brightness[1].max(), clip=True))
#t = [0,50,100]
#im = ax.imshow(brightness.reshape((npix,npix)), cmap=my_cmap, norm=LogNorm(vmin=0.1, vmax=20, clip=True))
#t = [0.1,0.5,1,5,10,20]

ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel(r'D Ly $\beta$'+'\n[R]', rotation=0, labelpad=25);

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
from matplotlib.colors import LogNorm,PowerNorm
# C = some matrix
f, ax = plt.subplots(1,dpi=250)

import copy
my_cmap = copy.copy(idl_colorbars.getcmap(49)) # copy the default cmap
my_cmap.set_bad((0,0,0))

toshow=D_col_dens[0].reshape((npix,npix))
logmin=np.nanmin(toshow)
toshow[toshow<logmin] = logmin

im = ax.imshow(toshow, cmap=my_cmap, norm=LogNorm()) 
# norm=PowerNorm(0.5, vmin=0.0, vmax=D_col_dens.max(), clip=True)) #
#t = [0,5,10,15,20]
#im = ax.imshow(brightness.reshape((npix,npix)), cmap=my_cmap, norm=LogNorm(vmin=0.1, vmax=20, clip=True))
#t = [0.1,0.5,1,5,10,20]

ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel(r'D col. dens.'+'\n'+r'[cm$^{-2}$]', rotation=0, labelpad=35);

In [ ]:
np.log10(2*test_nD*3595e5)

In [ ]:
np.log10(np.max(D_col_dens))

# Lyman Alpha / Beta Doublet Simulation

In [ ]:
test_nH

In [ ]:
pycorona.lyman_multiplet_generate_source_function(test_nH, 
                                                  test_Texo, 
                                                  '/home/mike/Desktop/check_atm.dat',
                                                  '/home/mike/Desktop/check_sfn.dat')
%time multiplet_brightness = pycorona.lyman_multiplet_brightness()

In [ ]:
multiplet_brightness.max()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
from matplotlib.colors import LogNorm,PowerNorm
# C = some matrix
f, ax = plt.subplots(1,dpi=250)

import copy
my_cmap = copy.copy(idl_colorbars.getcmap(109)) # copy the default cmap
my_cmap.set_bad((0,0,0))

im = ax.imshow(multiplet_brightness[0].reshape((npix,npix)), cmap=my_cmap, norm=PowerNorm(0.5,vmin=0.0, vmax=12, clip=True))
t = [0,5,10,15,20]
#im = ax.imshow(multiplet_brightness.reshape((npix,npix)), cmap=my_cmap, norm=LogNorm(vmin=0.1, vmax=20, clip=True))
#t = [0.1,0.5,1,5,10,20]
ax.set_title('Full Doublet Treatment')

ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel(r'H Ly $\alpha$'+'\n[kR]', rotation=0, labelpad=25);

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(multiplet_brightness[0].reshape((npix,npix))-brightness[0].reshape((npix,npix)))
plt.colorbar(im)

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(multiplet_brightness[0].reshape((npix,npix))/brightness[0].reshape((npix,npix))-1)
plt.colorbar(im)

This will plot Lyman beta if observation_fit computes it:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
from matplotlib.colors import LogNorm,PowerNorm
# C = some matrix
f, ax = plt.subplots(dpi=250)

import copy
my_cmap = copy.copy(idl_colorbars.getcmap(114)) # copy the default cmap
my_cmap.set_bad((0,0,0))

im = ax.imshow(multiplet_brightness[1].reshape((npix,npix))*1000, cmap=my_cmap, norm=PowerNorm(0.5,vmin=0.0, vmax=80, clip=True))
#t = [0,50,100]
#im = ax.imshow(multiplet_brightness.reshape((npix,npix)), cmap=my_cmap, norm=LogNorm(vmin=0.1, vmax=20, clip=True))
#t = [0.1,0.5,1,5,10,20]

ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel(r'H Ly $\beta$'+'\n[R]', rotation=0, labelpad=25);

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(1000*(multiplet_brightness[1].reshape((npix,npix))-brightness[1].reshape((npix,npix))))
plt.colorbar(im)

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(multiplet_brightness[1].reshape((npix,npix))/brightness[1].reshape((npix,npix))-1)
plt.colorbar(im)

# Lyman Alpha / Beta Singlet Simulation

In [ ]:
test_nH

In [ ]:
pycorona.lyman_singlet_generate_source_function(test_nH, 
                                                  test_Texo, 
                                                  '/home/mike/Desktop/check_atm.dat',
                                                  '/home/mike/Desktop/check_sfn.dat')
%time singlet_brightness = pycorona.lyman_singlet_brightness()

In [ ]:
singlet_brightness.max()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
from matplotlib.colors import LogNorm,PowerNorm
# C = some matrix
f, ax = plt.subplots(1,dpi=250)

import copy
my_cmap = copy.copy(idl_colorbars.getcmap(109)) # copy the default cmap
my_cmap.set_bad((0,0,0))

im = ax.imshow(singlet_brightness[0].reshape((npix,npix)), cmap=my_cmap, norm=PowerNorm(0.5,vmin=0.0, vmax=12, clip=True))
t = [0,5,10,15,20]
#im = ax.imshow(singlet_brightness.reshape((npix,npix)), cmap=my_cmap, norm=LogNorm(vmin=0.1, vmax=20, clip=True))
#t = [0.1,0.5,1,5,10,20]

ax.set_title('Singlet via Multiplet Code')
ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel(r'H Ly $\alpha$'+'\n[kR]', rotation=0, labelpad=25);

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(singlet_brightness[0].reshape((npix,npix))-brightness[0].reshape((npix,npix)))
ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel('SvM - S\n[kR]', rotation=0, labelpad=25);

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(singlet_brightness[0].reshape((npix,npix))/brightness[0].reshape((npix,npix))-1)
ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel('(SvM - S)/S\n[%]', rotation=0, labelpad=25);

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(multiplet_brightness[0].reshape((npix,npix))-singlet_brightness[0].reshape((npix,npix)))
plt.colorbar(im)

This will plot Lyman beta if observation_fit computes it:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
from matplotlib.colors import LogNorm,PowerNorm
# C = some matrix
f, ax = plt.subplots(dpi=250)

import copy
my_cmap = copy.copy(idl_colorbars.getcmap(114)) # copy the default cmap
my_cmap.set_bad((0,0,0))

im = ax.imshow(singlet_brightness[1].reshape((npix,npix))*1000, cmap=my_cmap, norm=PowerNorm(0.5,vmin=0.0, vmax=80, clip=True))
#t = [0,50,100]
#im = ax.imshow(singlet_brightness.reshape((npix,npix)), cmap=my_cmap, norm=LogNorm(vmin=0.1, vmax=20, clip=True))
#t = [0.1,0.5,1,5,10,20]

ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel(r'H Ly $\beta$'+'\n[R]', rotation=0, labelpad=25);

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(1000*(singlet_brightness[1].reshape((npix,npix))-brightness[1].reshape((npix,npix))))
plt.colorbar(im)

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(1000*(singlet_brightness[1].reshape((npix,npix)) - multiplet_brightness[1].reshape((npix,npix))))
plt.colorbar(im)

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(singlet_brightness[1].reshape((npix,npix))/brightness[1].reshape((npix,npix))-1)
plt.colorbar(im)

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(multiplet_brightness[0].reshape((npix,npix))-singlet_brightness[0].reshape((npix,npix)))
ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel('D - SvM\n[kR]', rotation=0, labelpad=25);

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(1000*(multiplet_brightness[1].reshape((npix,npix))-singlet_brightness[1].reshape((npix,npix))))
ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel('D - SvM\n[R]', rotation=0, labelpad=25);

In [ ]:
f, ax = plt.subplots(1,dpi=250)
im = ax.imshow(100*(multiplet_brightness[1].reshape((npix,npix))/singlet_brightness[1].reshape((npix,npix))-1))
ax.set_axis_off()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
f.colorbar(im, cax=cax)#, ticks=t, format='$%.2f$');
cax.set_ylabel('D/SvM - 1 \n[%]', rotation=0, labelpad=25);

# Source Function Inspection

In [ ]:
import tempfile

In [ ]:
fname = tempfile.NamedTemporaryFile()

In [ ]:
fname.name

In [ ]:
def sfn_readline(line):
    if not ":" in line:
        # this is a potential header, return without stripping
        return [line, []]
    
    name, data = line.split(":")
    return [name, np.fromstring(data.strip(),sep=" ")]

def sfn_dict_append(sfn, file):
    emission_dict_name=None
    variable_dict_name=None
    level_dict_name=None
    subdict=sfn
    
    for line in file:
        name, data = sfn_readline(line)

        # figure out where to place the data on this line

        # figure out the indent level (number of leading spaces / 2)
        indent_level = (len(name) - len(name.lstrip()))/2
        if indent_level==0 and len(data)==0:
            emission_dict_name = name.strip().replace('For ','')
            if emission_dict_name not in sfn.keys():
                sfn[emission_dict_name] = dict()
            emission_dict = sfn[emission_dict_name]
            subdict=emission_dict
            continue
        if indent_level==1:
            # only SZA lives at this level
            continue
        if indent_level==2 and len(data)==0:
            # density, optical depth, etc.
            variable_dict_name = name.strip()
            if variable_dict_name not in sfn[emission_dict_name].keys():
                sfn[emission_dict_name][variable_dict_name] = dict()
            subdict = sfn[emission_dict_name][variable_dict_name]
            continue
        if indent_level==2 and len(data)>0:
            subdict=emission_dict

        name = name.strip()
        if name in subdict.keys():
            if subdict[name].ndim==1:
                subdict[name] = subdict[name][np.newaxis,:]
            subdict[name] = np.append(subdict[name], [data], axis=0)
        else:
            subdict[name] = data

def get_source_function(nH, T):
    fname = tempfile.NamedTemporaryFile()
    pycorona.generate_source_function(nH, T, sourcefn_fname=fname.name)
    sourcefn_file=np.loadtxt(fname.name,dtype=str,delimiter="\n")
    sfn=dict()
    sfn_dict_append(sfn,sourcefn_file)
    return sfn

In [ ]:
pycorona.set_use_CO2_absorption(True)

In [ ]:
sfn=get_source_function(5e5, 200)

In [ ]:
def plot_H_sourcefn(sfn, x_alt=True):
    fig, ax = plt.subplots(ncols=2,figsize=(9,4),
                           sharex=True,sharey=True,
                           gridspec_kw={'wspace':0.15},
                           dpi=150)
    
    if x_alt:
        alt = 1e-5*(sfn['pts radii [cm]']-3395e5)
        x = [alt, alt, alt]
        ax[0].set_xlim([80,1e5])
        ax[1].set_xlim([80,1e5])
        ax[0].set_xscale('log')
        ax[0].set_xlabel('Altitude [km]')
    else:
        tau0 = [sfn['H Lyman alpha']['Species single scattering tau'][0],
                sfn['H Lyman beta']['Species single scattering tau'][0]]
        x = tau0
        ax[0].set_xscale('log')
        ax[1].set_xscale('log')
        ax[0].invert_xaxis()
        ax[1].invert_yaxis()
        #ax[0].set_xlim([1e0,1e-8])
        #ax[1].set_xlim([1e1,1e-7])
        ax[0].set_xlabel(r'Line Center $\tau$')
        
    
    cmap = idl_colorbars.getcmap(80,reverse=True)
    
    nsza = len(sfn['pts sza [rad]'])
    for isza in range(nsza):
        ax[0].plot(x[0], sfn['H Lyman alpha']['Source function'][isza], color=cmap(isza/(nsza-1)))
        ax[1].plot(x[1], sfn['H Lyman beta']['Source function'][isza], color=cmap(isza/(nsza-1)))
    
    ax[0].set_title('Lyman alpha')
    ax[1].set_title('Lyman beta')
    
    ax[0].set_ylabel('Upper State Density / Optically Thin Density')

    ax[0].set_ylim([0,2.75])
    
#     ax[1].set_ylim([5e-4,10])
#     ax[1].set_yscale('log')
    
    ax[1].set_ylim([0,2.5])
    #ax[1].set_yscale('log')

In [ ]:
plot_H_sourcefn(sfn, x_alt=False)

In [ ]:
cmap = idl_colorbars.getcmap(80,reverse=True)

fig, ax = plt.subplots(nrows=2, sharex=True)

nsza = len(sfn['pts sza [rad]'])
for isza in range(nsza):
    ax[0].plot(sfn['H Lyman beta']['Species single scattering tau'][0], 
             sfn['H Lyman beta']['Source function'][isza],
             color=cmap(isza/(nsza-1)));
    ax[0].plot(sfn['H Lyman beta']['Species single scattering tau'][0], 
             sfn['H Lyman beta']['Species single scattering source function S0'][isza],
             color=cmap(isza/(nsza-1)),
             lw=0.5);

thickthinratio = (sfn['H Lyman beta']['Source function'][0]
                  /
                  sfn['H Lyman beta']['Species single scattering source function S0'][0])    

for isza in range(nsza):
    ax[1].plot(sfn['H Lyman beta']['Species single scattering tau'][0], 
               thickthinratio,
               color=cmap(0));

ax[0].set_xscale('log')

In [ ]:
1/0.12

In [ ]:
thickthinratio

In [ ]:
plot_H_sourcefn(sfn)

In [ ]:
(sc_mso, fov_mso) = np.load('/home/mike/Documents/EMM/emm_sys015_limb_geom.npy')
sys15_alt = np.load('/home/mike/Documents/EMM/emm_sys015_limb_alt.npy')

In [ ]:
fov_mso.shape

In [ ]:
pycorona.add_observation(sc_mso,fov_mso)
time.sleep(1)

In [ ]:
def get_model_brightness(nH, Texo):
    # get H brightness
    pycorona.generate_source_function(nH,Texo)
    pycorona.set_g_factor([1.22848e-3, 2.05e-6])
    H_brightness = 1000*pycorona.brightness()
    
    lyman_beta_brightness = np.reshape(H_brightness[1], (4,360,128))
    
    return np.mean(lyman_beta_brightness, axis=(0,2))

In [ ]:
lyb_brightness = [get_model_brightness(nH,200) for nH in np.logspace(-1,3,num=9,base=2)*1e5]

In [ ]:
8*360*128

In [ ]:
lyb_brightness = [get_model_brightness(2e5,Texo) for Texo in np.linspace(100,300,9)]

In [ ]:
fig, ax = plt.subplots(1, figsize=(3,4), dpi=150)
[ax.plot(b,sys15_alt, color='C0') for b in lyb_brightness]
ax.set_ylim([136,1000]);

In [ ]:
sfns = [get_source_function(nH,200) for nH in np.logspace(-1,3,num=9,base=2)*1e5]

In [ ]:
plt.plot([(s['pts radii [cm]']/1e5-3395) for s in sfns]);
plt.ylim([80,500])

In [ ]:
plt.plot([(s['pts radii [cm]']/1e5-3395) for s in sfns]);
plt.ylim([80,200])

# Simulated EMM OS2 geometry

In [ ]:
import numpy as np
import spiceypy as spice
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches':'tight'}
plt.rcParams['figure.dpi']=300

In [ ]:
spice.kclear()
spice.furnsh(['/home/mike/Documents/EMM/science_closure_corona_figure/kernels/de430.bsp',
              '/home/mike/Documents/EMM/science_closure_corona_figure/kernels/naif0011.tls',
              '/home/mike/Documents/EMM/science_closure_corona_figure/kernels/pck00010.tpc',
              '/home/mike/Documents/EMM/science_closure_corona_figure/kernels/mar097.bsp',
              '/home/mike/Documents/EMM/science_closure_corona_figure/kernels/DE403-MASSES.TPC',
              '/home/mike/Documents/MAVEN/IUVS/iuvs-itf-sw/anc/spice/mvn/fk/maven_v09.tf',
              '/home/mike/Documents/EMM/science_closure_corona_figure/kernels/Amal.bsp'])
#now info about EMM orbit is stored in spice body number -200000

In [ ]:
#let's pick perihelion for an example image
etstart=spice.str2et('2022 Jun 22')
amalstate=spice.spkezr('-200000',etstart,'MAVEN_MME_2000','NONE','MARS')[0]

In [ ]:
marsmu=spice.bodvrd('MARS','GM',1)[1][0]

In [ ]:
elements=spice.oscltx(amalstate,etstart,marsmu)
emm_ecc=elements[1]
emm_inc=elements[2]
emm_semimajor_axis=elements[-2]
emm_orbit_period=elements[-1]

In [ ]:
#find the location of periapsis
#adapted from the spicepy gfposc tutorial
MAXIVL = 1000
MAXWIN = 2 * MAXIVL
cnfine = spice.stypes.SPICEDOUBLE_CELL(2)
spice.wninsd(etstart,etstart+emm_orbit_period,cnfine)
riswin = spice.stypes.SPICEDOUBLE_CELL( MAXWIN )

In [ ]:
spice.gfposc('-200000','MAVEN_MSO','NONE','MARS',
             'LATITUDINAL','RADIUS','ABSMIN',0.0,
             0.0,emm_orbit_period/2, MAXIVL, cnfine, riswin)
emm_peritime=spice.wnfetd(riswin,0)[0]
spice.timout(emm_peritime,'YYYY MON DD HR:MN:SC.### (TDB) ::TDB')

In [ ]:
peri_state=spice.spkezr('-200000',emm_peritime,'MAVEN_MME_2000','NONE','MARS')[0]
peri_elements=spice.oscltx(peri_state,emm_peritime,marsmu)

In [ ]:
peri_state_mso=spice.spkezr('-200000',emm_peritime,'MAVEN_MSO','NONE','MARS')[0]
peri_vec=-peri_state_mso[:3]/np.linalg.norm(peri_state_mso[:3])

In [ ]:
orbit_subdivisions=1000
etlist=emm_peritime+1.025*emm_orbit_period*np.linspace(0,1,num=orbit_subdivisions)

In [ ]:
statelist=spice.spkezr('-200000',etlist,'MAVEN_MSO','NONE','MARS')[0]
sundirection=spice.spkezr('SUN',etlist[0],'MAVEN_MSO','NONE','MARS')[0]

In [ ]:
normvecs=[]
for s in statelist:
    c=np.cross(s[:3],s[-3:])
    normvecs.append(c/np.linalg.norm(c))
normal_vec=np.mean(normvecs,axis=0)
cross_vec=np.cross(normal_vec,-peri_vec)

sun_in_orb_vec=sundirection[:3]-np.dot(sundirection[:3],normal_vec)*normal_vec
sun_in_orb_vec=sun_in_orb_vec/np.linalg.norm(sun_in_orb_vec)
sun_cross_vec=np.cross(normal_vec,sun_in_orb_vec)

In [ ]:
x_vec=sun_in_orb_vec
y_vec=sun_cross_vec

xyz=[]
stateangles=[]
for s in statelist:
    xyz.append([np.dot(s[:3],x_vec),np.dot(s[:3],y_vec),np.dot(s[:3],normal_vec)])
    orbcoords=xyz[-1][:2]/np.linalg.norm(xyz[-1][:2])
    stateangles.append(np.arctan2(orbcoords[0],orbcoords[1]))
xyz=np.array(xyz)

sunxyz=[np.dot(sundirection[:3],x_vec),np.dot(sundirection[:3],y_vec),np.dot(sundirection[:3],normal_vec)]
sunxyz=sunxyz/np.linalg.norm(sunxyz)

In [ ]:
def draw_orbit(ax):
    ax.plot(*(np.transpose(xyz[:,:2])))
    ax.set_aspect('equal')
    rmars=np.mean(spice.bodvrd('MARS','RADII',3)[1])
    mars = plt.Circle((0, 0), rmars, color='#BC2A3A')
    ax.add_artist(mars)

    sunarrowend=(sunxyz[0]*2e4,sunxyz[1]*2e4)
    sunarrowstart=(sunxyz[0]*1.5*rmars,sunxyz[1]*1.5*rmars)
    ax.annotate('',
                xy=sunarrowstart,xytext=sunarrowend,
                arrowprops=dict(facecolor='black',edgecolor='black',width=0,headwidth=5,headlength=5));
    ax.text(2e4,4e3,'Sunlight',ha='right',va='bottom',fontsize=6);

In [ ]:
trueanomalylist=[]
for s,t in zip(statelist,etlist):
    trueanomalylist.append(spice.oscltx(s,t,marsmu)[-3])

In [ ]:
orbit_plane_angles=[]
for s in statelist:
    p=s[:3]/np.linalg.norm(s[:3])
    x=np.dot(p,-peri_vec)
    y=np.dot(p,cross_vec)
    orbit_plane_angles.append(np.arctan2(y,x))

For some reason the true anomaly reported by spice is not symmetric around the orbit??

In [ ]:
#now let's find the values we're interested in
nimgs=6
view_peri_angles=np.linspace(0,2*np.pi,nimgs+1)[:nimgs]

view_peri_angles=[a if a<np.pi else a-2*np.pi for a in view_peri_angles]

view_et=[etlist[(np.abs(orbit_plane_angles-v)).argmin()] for v in view_peri_angles]

In [ ]:
view_state=spice.spkezr('-200000',view_et,'MAVEN_MSO','NONE','MARS')[0]
view_state=np.array(view_state)

In [ ]:
viewxyz=[]
for s in view_state:
    viewxyz.append([np.dot(s[:3],x_vec),np.dot(s[:3],y_vec),np.dot(s[:3],normal_vec)])
viewxyz=np.array(viewxyz)
view_sun_angles=[np.arctan2(p[1],p[0]) for p in viewxyz]

In [ ]:
def lookdir(az,ax,center_dir,right_dir,up_dir):
    angle1=R.from_rotvec(-az*up_dir).as_matrix()
    angle2=R.from_rotvec(ax*right_dir).as_matrix()
    return np.array([*map(lambda x:np.matmul(x,center_dir),
                          [*map(np.matmul,angle1,angle2)])])

In [ ]:
def get_obs_vectors(pos,angle=30,npix=600):
    #pos is the spacecraft position in cm
    posdir=pos/np.linalg.norm(pos)

    right_dir=sundirection[:3]/np.linalg.norm(sundirection[:3])
    right_dir=right_dir-np.dot(right_dir,posdir)*posdir
    right_dir=right_dir/np.linalg.norm(right_dir)
    
    up_dir=[0,0,1]
    up_dir=up_dir-np.dot(up_dir,posdir)*posdir-np.dot(right_dir,up_dir)*right_dir
    up_dir=up_dir/np.linalg.norm(up_dir)
    #ensure that the illuminated portion of the atmosphere is to the right, 
    #checking the MSO y-component to see which side of the planet we're on
    if posdir[1]>0:
        up_dir=-up_dir    
    
    #print(np.dot(posdir,right_dir))
    #print(np.dot(posdir,up_dir))
    #print(np.dot(up_dir,right_dir))

    center_dir=-posdir
    lookangles=np.linspace(-angle*np.pi/180.,angle*np.pi/180.,npix)
    
    anglepairs=np.array([([ax],[az]) for az in lookangles for ax in lookangles])
    all_lookangles=lookdir(anglepairs[:,0],anglepairs[:,1],center_dir,right_dir,up_dir)
    all_pos = np.array([pos for i in all_lookangles])
    return (all_pos, all_lookangles)

In [ ]:
def fig2rgb_array(fig):
    fig.canvas.draw()
    buf = fig.canvas.tostring_rgb()
    ncols, nrows = fig.canvas.get_width_height()
    return np.frombuffer(buf, dtype=np.uint8).reshape(nrows, ncols, 3)

In [ ]:
import matplotlib.lines as mlines

def place_obsimage(ax,img,xyorb_loc,xyimg_loc,size=11000,align=(0,0),widthratio=2):
    
    x0=xyimg_loc[0]-widthratio*size*(1-align[0])/2
    x1=xyimg_loc[0]+widthratio*size*(1+align[0])/2
    y0=xyimg_loc[1]-size*(1-align[1])/2
    y1=xyimg_loc[1]+size*(1+align[1])/2
    
    
    l = mlines.Line2D([xyorb_loc[0],xyimg_loc[0]], [xyorb_loc[1],xyimg_loc[1]],zorder=10,lw=0.5)
    ax.add_line(l)
#    print((x0,x1,y0,y1))
    
    ax.imshow(img,extent=(x0,x1,y0,y1),zorder=50)

In [ ]:
def place_orbit_images(ax,viewxyz,viewimgs,imgsize=50000,label_emissions=True,widthratio=2):
    posangle=np.mod((np.array(view_peri_angles)+np.pi)+np.pi/6,2*np.pi)-np.pi
    img_pos_idx=[np.abs(posangle-a).argmin() for a in view_sun_angles]
    posdist=[12e4,5.5e4]
    img_pos=np.array([[posdist[0]*np.cos(a),posdist[1]*np.sin(a)] for a in posangle])
    img_pos[1][1]+=2.5e4
    img_pos[-2][1]-=2.5e4
    for idx,(orb_pos,pos_idx) in enumerate(zip(viewxyz,img_pos_idx)):
        place_obsimage(ax,viewimgs[idx],orb_pos[:2],img_pos[pos_idx],size=imgsize,widthratio=widthratio)
    if label_emissions:
        ax.text(-14e4,5.5e4,"Ly "+r"$\beta$")
        ax.text(-9e4,5.5e4,"Ly "+r"$\alpha$")

# O 102.6 nm source function sensitivity studies

In [ ]:
pycorona.O_1026_generate_source_function(1e7,200,1.69e-3)

In [ ]:
def get_O_1026_brightness(pos,angle=30,npix=300):
    all_pos,all_lookangles=get_obs_vectors(pos,angle=angle,npix=npix)
    pycorona.add_observation(all_pos,all_lookangles)
    return pycorona.O_1026_brightness()

In [ ]:
def plot_O_1026_brightness(brightness,dim=None,show=True):
    if dim==None:
        npix=int(np.sqrt(len(brightness[0])))
        dim=(npix,npix)
    
    f, (ax_im) = plt.subplots(ncols=1, figsize=(2,2), dpi=200)
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    plt.margins(0,0)

    import copy
    o_1026_cmap = copy.copy(idl_colorbars.getcmap(105)) # copy the default cmap
    o_1026_cmap.set_bad((0,0,0))

    all_lines_brightness = 1000*np.sum(brightness,axis=0)
    
    im = ax_im.imshow(all_lines_brightness.reshape(dim), cmap=o_1026_cmap, norm=PowerNorm(1,vmin=0.0, vmax=40, clip=True))
    ax_im.set_axis_off()
    ax_im.xaxis.set_major_locator(plt.NullLocator())
    ax_im.yaxis.set_major_locator(plt.NullLocator())
   
    f.canvas.draw()
    rgb_array=fig2rgb_array(f)
    
    if show==False:
        plt.close()
    
    return rgb_array

In [ ]:
o_1026_brightness_view=[]
for s in view_state:
    o_1026_brightness_view.append(get_O_1026_brightness(s[:3]*1e5,angle=12.5))

In [ ]:
np.max(1000*np.sum(o_1026_brightness_view,axis=1))

In [ ]:
o_1026_viewimgs=[]
for im in o_1026_brightness_view:
    o_1026_viewimgs.append(plot_O_1026_brightness(im,show=False));

In [ ]:
fig, ax_sym = plt.subplots(1,dpi=200,)
fig.patch.set_facecolor('white')

draw_orbit(ax_sym)
ax_sym.scatter(viewxyz[:,0],viewxyz[:,1])

place_orbit_images(ax_sym,viewxyz,o_1026_viewimgs,widthratio=1,label_emissions=False)

ax_sym.autoscale(True)
ax_sym.set_axis_off()

fig.canvas.draw()
plt.tight_layout()

In [ ]:
solar_lyman_beta_flux_Hz = 1.69e-3

def get_source_function_O_1026(nO, T, fname=None):
    if fname is None:
        fname = tempfile.NamedTemporaryFile().name
        pycorona.O_1026_generate_source_function(nO, T, solar_lyman_beta_flux_Hz, sourcefn_fname=fname)
    sourcefn_file=np.loadtxt(fname,dtype=str,delimiter="\n")
    sfn=dict()
    sfn_dict_append(sfn,sourcefn_file)
    return sfn

In [ ]:
O_1026_sfn=get_source_function_O_1026(2e7,200)
O_1026_sfn['O_1026']['Species density [cm-3]']['lower state 2'][0]

In [ ]:
def get_O_1026_relative_source_function(O_density, T_exo, fname=None):
    opt_thin_upper_state_1_fraction = solar_lyman_beta_flux_Hz*2.00e-4*2.647e-2/(1.0697e8)
    opt_thin_upper_state_2_fraction = solar_lyman_beta_flux_Hz*3.01e-3*2.647e-2/(1.0714e8)
    opt_thin_upper_state_3_fraction = solar_lyman_beta_flux_Hz*1.69e-2*2.647e-2/(1.0750e8)
    
    O_1026_sfn = get_source_function_O_1026(O_density, T_exo, fname=fname)
    
    lower_state_2 = O_1026_sfn['O_1026']['Species density [cm-3]']['lower state 2']
    
    opt_thin_upper_state_1 = lower_state_2*opt_thin_upper_state_1_fraction;
    opt_thin_upper_state_2 = lower_state_2*opt_thin_upper_state_2_fraction;
    opt_thin_upper_state_3 = lower_state_2*opt_thin_upper_state_3_fraction;
    
    upper_state_rel_1 = O_1026_sfn['O_1026']['Source function']['upper state 0']/opt_thin_upper_state_1
    upper_state_rel_2 = O_1026_sfn['O_1026']['Source function']['upper state 1']/opt_thin_upper_state_2
    upper_state_rel_3 = O_1026_sfn['O_1026']['Source function']['upper state 2']/opt_thin_upper_state_3
    
    overall_upper_state_enhancement = ((O_1026_sfn['O_1026']['Source function']['upper state 0']
                                       +O_1026_sfn['O_1026']['Source function']['upper state 1']
                                       +O_1026_sfn['O_1026']['Source function']['upper state 2'])
                                       /(opt_thin_upper_state_1
                                         +opt_thin_upper_state_2
                                         +opt_thin_upper_state_3))
    
    O_1026_sfn['O_1026']['upper state relative densities'] = np.array([upper_state_rel_1,
                                                                       upper_state_rel_2,
                                                                       upper_state_rel_3])
    
    O_1026_sfn['O_1026']['overall upper state enhancement'] = overall_upper_state_enhancement
    
    return O_1026_sfn

In [ ]:
sfn=get_O_1026_relative_source_function(2e7,200)

In [ ]:
sfn['radial boundaries [cm]']

In [ ]:
sfn['O_1026']['Temperature [K]'][0]

In [ ]:
sfn['O_1026']['Species density [cm-3]']['lower state 2'][0]

In [ ]:
sfn['O_1026']['Species single scattering tau']['line 4'][0]

In [ ]:
sfn['O_1026']['Species single scattering source function S0']['upper state 2'][0]

In [ ]:
sfn['O_1026']['Source function']['upper state 0'][0]

In [ ]:
plt.plot(1e-5*(sfn['pts radii [cm]']-3395e5),sfn['O_1026']['overall upper state enhancement'][0]);
plt.axhline(1.15)

In [ ]:
sfn['O_1026'].keys()

In [ ]:
fig, ax = plt.subplots(1)

ax.plot(sfn['O_1026']['Species single scattering source function S0']['upper state 0'][0], sfn['O_1026']['Species single scattering tau']['line 5'][0])
ax.plot(sfn['O_1026']['Species single scattering source function S0']['upper state 1'][0], sfn['O_1026']['Species single scattering tau']['line 5'][0])
ax.plot(sfn['O_1026']['Species single scattering source function S0']['upper state 2'][0], sfn['O_1026']['Species single scattering tau']['line 5'][0])
ax.set_yscale('log')
ax.invert_yaxis()
ax.set_xscale('log')
ax.axhline(1, color='0.5')

In [ ]:
fig, ax = plt.subplots(ncols=2, sharey=True)

ax[0].plot(sfn['O_1026']['Source function']['upper state 0'][0]/(sfn['O_1026']['Source function']['upper state 0'][0]
                                                              +sfn['O_1026']['Source function']['upper state 1'][0]
                                                              +sfn['O_1026']['Source function']['upper state 2'][0]),
        sfn['O_1026']['Species single scattering tau']['line 5'][0])
ax[0].plot(sfn['O_1026']['Source function']['upper state 1'][0]/(sfn['O_1026']['Source function']['upper state 0'][0]
                                                              +sfn['O_1026']['Source function']['upper state 1'][0]
                                                              +sfn['O_1026']['Source function']['upper state 2'][0]),
        sfn['O_1026']['Species single scattering tau']['line 5'][0])
ax[0].plot(sfn['O_1026']['Source function']['upper state 2'][0]/(sfn['O_1026']['Source function']['upper state 0'][0]
                                                              +sfn['O_1026']['Source function']['upper state 1'][0]
                                                              +sfn['O_1026']['Source function']['upper state 2'][0]),
        sfn['O_1026']['Species single scattering tau']['line 5'][0])
ax[0].set_yscale('log')
ax[0].invert_yaxis()
ax[0].axhline(1, color='0.5')

ax[1].plot(sfn['O_1026']['Source function']['upper state 0'][0], sfn['O_1026']['Species single scattering tau']['line 5'][0])
ax[1].plot(sfn['O_1026']['Source function']['upper state 1'][0], sfn['O_1026']['Species single scattering tau']['line 5'][0])
ax[1].plot(sfn['O_1026']['Source function']['upper state 2'][0], sfn['O_1026']['Species single scattering tau']['line 5'][0])


ax[1].set_xscale('log')
ax[1].axhline(1, color='0.5')
ax[1].set_xlim([1e-16,1e-6])

In [ ]:
plt.plot(1e-5*(sfn['pts radii [cm]']-3395e5), sfn['O_1026']['Species single scattering tau']['line 3'][0], marker='.')
plt.plot(1e-5*(sfn['pts radii [cm]']-3395e5), sfn['O_1026']['Species single scattering tau']['line 4'][0], marker='.')
plt.plot(1e-5*(sfn['pts radii [cm]']-3395e5), sfn['O_1026']['Species single scattering tau']['line 5'][0], marker='.')
plt.plot(1e-5*(sfn['pts radii [cm]']-3395e5), sfn['O_1026']['Absorber single scattering tau']['line 0'][0], marker='.')
plt.yscale('log')
plt.axhline(1,color='0.5')
plt.axhline(10,color='0.5')
plt.axvline(200,color='0.5')

In [ ]:
sfn['O_1026']['Absorber single scattering tau']['line 0'][0]

In [ ]:
cmap = idl_colorbars.getcmap(80,reverse=True)

fig, ax = plt.subplots(nrows=2, sharex=True)

nsza = len(sfn['pts sza [rad]'])
for isza in range(nsza):
    ax[0].plot(sfn['O_1026']['Species single scattering tau']['line 5'][0], 
               sfn['O_1026']['Source function']['upper state 2'][isza],
               color=cmap(isza/(nsza-1)));
    ax[0].plot(sfn['O_1026']['Species single scattering tau']['line 5'][0], 
               sfn['O_1026']['Species single scattering source function S0']['upper state 2'][isza],
               color=cmap(isza/(nsza-1)),
               lw=0.5);
    
thickthinratio = (sfn['O_1026']['Source function']['upper state 2'][0]
                  /
                  sfn['O_1026']['Species single scattering source function S0']['upper state 2'][0])    

for isza in range(nsza):
    ax[1].plot(sfn['O_1026']['Species single scattering tau']['line 5'][0], 
               thickthinratio,
               color=cmap(0));
    
ax[0].set_xscale('log')
ax[1].set_ylim(1,4)

In [ ]:
thickthinratio

In [ ]:
import idl_colorbars

In [ ]:
def plot_O_1026_sourcefn(sfn, x_alt=True):
    
    fig, ax = plt.subplots(ncols=3,figsize=(13,4),sharex=False,sharey=True,gridspec_kw={'wspace':0.15})
    
    if x_alt:
        alt = 1e-5*(sfn['pts radii [cm]']-3395e5)
        x = [alt, alt, alt]
#         ax[0].set_xlim([100,700])
#         ax[1].set_xlim([100,700])
#         ax[2].set_xlim([100,700])
        ax[0].set_xlabel('Altitude [km]')
    else:
        tau0 = [sfn['O_1026']['Species single scattering tau']['line 3'][0],
                sfn['O_1026']['Species single scattering tau']['line 4'][0],
                sfn['O_1026']['Species single scattering tau']['line 5'][0]]
        x = tau0
        #x = np.repeat(np.sum(x, axis=0)[np.newaxis,:],3, axis=0)
        ax[0].set_xscale('log')
        ax[1].set_xscale('log')
        ax[2].set_xscale('log')
        ax[0].set_xlim([1e0,1e-8])
        ax[1].set_xlim([1e1,1e-7])
        ax[2].set_xlim([1e2,1e-6])
        ax[0].set_xlabel(r'Line Center $\tau$')
        
    
    cmap = idl_colorbars.getcmap(80,reverse=True)
    
    nsza = len(sfn['pts sza [rad]'])
    for isza in range(nsza):
        ax[0].plot(x[0], sfn['O_1026']['upper state relative densities'][0][isza], color=cmap(isza/(nsza-1)))
        ax[1].plot(x[1], sfn['O_1026']['upper state relative densities'][1][isza], color=cmap(isza/(nsza-1)))
        ax[2].plot(x[2], sfn['O_1026']['upper state relative densities'][2][isza], color=cmap(isza/(nsza-1)))
    
    ax[0].set_title('upper state J=1')
    ax[1].set_title('J=2')
    ax[2].set_title('J=3')
    
    ax[0].set_ylabel('Upper State Density / Optically Thin Density')
    

    ax[0].set_ylim([0,2.75])

In [ ]:
plot_O_1026_sourcefn(sfn, x_alt=False)

In [ ]:
plot_O_1026_sourcefn(sfn, x_alt=True)

In [ ]:
plot_O_1026_sourcefn(get_O_1026_relative_source_function(2e7,200), x_alt=True)

In [ ]:
plot_O_1026_sourcefn(get_O_1026_relative_source_function(2e7,200, fname='/home/mike/Documents/Mars/3D_planetary_RT_model/test/test_source_function_O_1026_40x20x7x12.dat'), x_alt=True)

In [ ]:
plot_O_1026_sourcefn(get_O_1026_relative_source_function(2e7,200, fname='/home/mike/Documents/Mars/3D_planetary_RT_model/test/test_source_function_O_1026_80x80x7x12.dat'), x_alt=True)

In [ ]:
plot_O_1026_sourcefn(get_O_1026_relative_source_function(2e7,200, fname='/home/mike/Documents/Mars/3D_planetary_RT_model/test/test_source_function_O_1026_40x20x21x36.dat'), x_alt=True)

In [ ]:
pp_sfn = get_O_1026_relative_source_function(2e7, 200, fname='/home/mike/Documents/Mars/3D_planetary_RT_model/test/test_source_function_O_1026_40x6.dat')

plt.plot(1e-5*(sfn['pts radii [cm]']-3395e5),pp_sfn['O_1026']['upper state relative densities'][0]);
plt.plot(1e-5*(sfn['pts radii [cm]']-3395e5),pp_sfn['O_1026']['upper state relative densities'][1]);
plt.plot(1e-5*(sfn['pts radii [cm]']-3395e5),pp_sfn['O_1026']['upper state relative densities'][2]);

In [ ]:
def get_O_1026_relative_upper_state_density():
    sfn_reference = get_O_1026_relative_source_function(2e7,200)
    reference_alt = 1e-5*(sfn_reference['pts radii [cm]']-3395e5)
    reference_density = sfn_reference['O_1026']['Source function']['upper state 2'][0]
    
    density_comp_alt = []
    density_comp = []
    for O_density in [0.5e7, 1e7, 4e7, 8e7]:
        sfn=get_O_1026_relative_source_function(O_density,200)
        density_comp_alt.append(1e-5*(sfn['pts radii [cm]']-3395e5))
        density_comp.append(sfn['O_1026']['Source function']['upper state 2'][0])
        
    temp_comp_alt = []
    temp_comp = []
    for O_temp in [100,150,250,300]:
        sfn=get_O_1026_relative_source_function(2e7,O_temp)
        temp_comp_alt.append(1e-5*(sfn['pts radii [cm]']-3395e5))
        temp_comp.append(sfn['O_1026']['Source function']['upper state 2'][0])
        
    return [[reference_alt, reference_density], [density_comp_alt, density_comp], [temp_comp_alt, temp_comp]]

In [ ]:
comp_data = get_O_1026_relative_upper_state_density()

In [ ]:
fig, ax = plt.subplots(nrows=2,figsize=(6,5),sharex=True,sharey=True,gridspec_kw={'hspace':0.075})
ax[0].set_xlim([100,700])
ax[0].plot(comp_data[0][0],1e8*comp_data[0][1],color='gray')
ax[1].plot(comp_data[0][0],1e8*comp_data[0][1],color='gray')

ax[0].set_title('Excited State Density, Subsolar J=3')
ax[0].set_ylabel('Density [10$^{-8}$ cm$^{-3}$]')
ax[1].set_xlabel('Altitude [km]')

dens_colors = [idl_colorbars.getcmap(54)(x) for x in [0.2,0.3,0.7,0.8]]
for alt, dens, color in zip(comp_data[1][0],comp_data[1][1], dens_colors):
    ax[0].plot(alt,1e8*dens,color=color)

ax[0].text(x=300,y=3.8,s='O Exobase Density [10$^7$ cm$^{-3}$]\n0.5   1.0   2.0   4.0   8.0')

temp_colors = [idl_colorbars.getcmap(70)(x) for x in [0.8,0.7,0.3,0.2]]
for alt, dens, color in zip(comp_data[2][0],comp_data[2][1], temp_colors):
    ax[1].plot(alt,1e8*dens,color=color)

ax[1].text(x=300,y=3.8,s='Exobase Temperature\n100K   150K   200K   250K   300K');

    
#ax.set_xscale('log')
#ax.set_yscale('log')
#ax.set_ylim([1e-12,1e-7]);